In [2]:
import cv2
import matplotlib.pyplot as plt
import mediapipe as mp
import os


In [3]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawings = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

FACEMESH_LIPS = [(61, 146), (146, 91), (91, 181), (181, 84), (84, 17),
                 (17, 314), (314, 405), (405, 321), (321, 375),
                 (375, 291), (61, 185), (185, 40), (40, 39), (39, 37),
                 (37, 0), (0, 267),
                 (267, 269), (269, 270), (270, 409), (409, 291),
                 (78, 95), (95, 88), (88, 178), (178, 87), (87, 14),
                 (14, 317), (317, 402), (402, 318), (318, 324),
                 (324, 308), (78, 191), (191, 80), (80, 81), (81, 82),
                 (82, 13), (13, 312), (312, 311), (311, 310),
                 (310, 415), (415, 308)]

FACEMESH_LEFT_EYE = [(263, 249), (249, 390), (390, 373), (373, 374),
                     (374, 380), (380, 381), (381, 382), (382, 362),
                     (263, 466), (466, 388), (388, 387), (387, 386),
                     (386, 385), (385, 384), (384, 398), (398, 362)]

FACEMESH_LEFT_IRIS = [(474, 475), (475, 476), (476, 477),
                      (477, 474)]

FACEMESH_LEFT_EYEBROW = [(276, 283), (283, 282), (282, 295),
                         (295, 285), (300, 293), (293, 334),
                         (334, 296), (296, 336)]

FACEMESH_RIGHT_EYE = [(33, 7), (7, 163), (163, 144), (144, 145),
                      (145, 153), (153, 154), (154, 155), (155, 133),
                      (33, 246), (246, 161), (161, 160), (160, 159),
                      (159, 158), (158, 157), (157, 173), (173, 133)]

FACEMESH_RIGHT_EYEBROW = [(46, 53), (53, 52), (52, 65), (65, 55),
                          (70, 63), (63, 105), (105, 66), (66, 107)]

FACEMESH_RIGHT_IRIS = [(469, 470), (470, 471), (471, 472),
                       (472, 469)]

TrainingDataDirectory = "../DATA/Train/"
TestingDataDirectory = "../DATA/Test/"
Classes = [x for x in os.listdir(TrainingDataDirectory)]
UPPER_REF = 10
LOWER_REF = 152

In [4]:
def FACEMESH_COMPRESSOR(FACEMESH, n):
    TEMP_FACEMESH = []
    FINAL_FACEMESH = []
    for i in range(len(FACEMESH)):
        TEMP_FACEMESH.append(FACEMESH[i][0])
        TEMP_FACEMESH.append(FACEMESH[i][1])
    TEMP_FACEMESH = list(dict.fromkeys(TEMP_FACEMESH))
    for i in range(0, len(TEMP_FACEMESH), n):
        FINAL_FACEMESH.append(TEMP_FACEMESH[i])
    return FINAL_FACEMESH

In [5]:
FOREHEAD_TO_LEFT_EYEBROW = [(UPPER_REF,46),(UPPER_REF,63),(UPPER_REF,52),(UPPER_REF,66),(UPPER_REF,55)]
FOREHEAD_TO_RIGHT_EYEBROW = [(UPPER_REF,285),(UPPER_REF,296),(UPPER_REF,282),(UPPER_REF,293),(UPPER_REF,276)]
LEFT_EYE_TO_LEFT_EYEBROW = [(33,46),(161,63),(159,52),(157,66),(133,55)]
RIGHT_EYE_TO_RIGHT_EYEBROW = [(362,285),(384,296),(386,282),(388,293),(263,276)]
LEFT_EYEBROW_TO_RIGHT_EYEBROW = [(55,285)]
LEFT_EYE_HEIGHT = [(161,163),(159,145),(157,154)]
LEFT_EYE_WIDTH = [(33,133)]
RIGHT_EYE_HEIGHT = [(384,381),(386,374),(388,390)]
RIGHT_EYE_WIDTH = [(362,263)]
MOUTH_LEFT_CORNER_TO_LOWER_LEFT_EYE = [(61,33),(61,163),(61,145),(61,154),(61,133)]
MOUTH_RIGHT_CORNER_TO_LOWER_RIGHT_EYE = [(291,362),(291,381),(291,374),(291,390),(291,263)]
MOUTH_HEIGHT = [(40,91),(37,84),(314,267),(321,270)]
MOUTH_WIDTH = [(61,291)]
CHIN_TO_LOWER_LIP = [(LOWER_REF,61),(LOWER_REF,91),(LOWER_REF,84),(LOWER_REF,314),(LOWER_REF,321),(LOWER_REF,291)]
LOWER_NOSE_TO_UPPER_LIP = [(1,291),(1,40),(1,37),(1,267),(1,270)]
NOSE_HEIGHT=[(1,6)]
UPPER_NOSE_TO_INNER_EYES = [(6,133),(6,362)]
UPPER_NOSE_TO_INNER_EYEBROWS = [(6,55),(6,285)]

DISTANCES = FOREHEAD_TO_LEFT_EYEBROW+FOREHEAD_TO_RIGHT_EYEBROW+LEFT_EYE_TO_LEFT_EYEBROW+RIGHT_EYE_TO_RIGHT_EYEBROW+LEFT_EYEBROW_TO_RIGHT_EYEBROW+LEFT_EYE_HEIGHT+LEFT_EYE_WIDTH+RIGHT_EYE_HEIGHT+RIGHT_EYE_WIDTH+MOUTH_LEFT_CORNER_TO_LOWER_LEFT_EYE+MOUTH_RIGHT_CORNER_TO_LOWER_RIGHT_EYE+MOUTH_HEIGHT+MOUTH_WIDTH+CHIN_TO_LOWER_LIP+LOWER_NOSE_TO_UPPER_LIP+NOSE_HEIGHT+UPPER_NOSE_TO_INNER_EYES+UPPER_NOSE_TO_INNER_EYEBROWS

print(len(DISTANCES))


60


In [6]:
from math import sqrt
def calculate_distance(a,b):
    return sqrt((a.x-b.x)*(a.x-b.x) + (a.y-b.y)*(a.y-b.y))

### CAM WITH LANDMARKS

In [7]:
# drawing_spec = mp_drawings.DrawingSpec(thickness=1, circle_radius=1)
# cap = cv2.VideoCapture(0)
# with mp_face_mesh.FaceMesh(
#     max_num_faces=1,
#     refine_landmarks=True,
#     min_detection_confidence=0.5,
#     min_tracking_confidence=0.5) as face_mesh:
#   while cap.isOpened():
#     success, annotated_image = cap.read()
#     height,width,_= annotated_image.shape
#     if not success:
#       print("Ignoring empty camera frame.")
#       # If loading a video, use 'break' instead of 'continue'.
#       continue
#
#     # To improve performance, optionally mark the image as not writeable to
#     # pass by reference.
#     annotated_image.flags.writeable = False
#     annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
#     results = face_mesh.process(annotated_image)
#
#     # Draw the face mesh annotations on the image.
#     annotated_image.flags.writeable = True
#     annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)
#     if results.multi_face_landmarks:
#       for face_landmarks in results.multi_face_landmarks:
#           for i in landmarks:
#               x=int(face_landmarks.landmark[i].x * width)
#               y=int(face_landmarks.landmark[i].y * height)
#               cv2.circle(img=annotated_image,center=(x,y),radius=2,color=(0,0,255),thickness=-1)
#               cv2.putText(img=annotated_image,text=str(i),org=(x,y+3),fontFace=cv2.FONT_HERSHEY_DUPLEX,fontScale=0.3,color=(255,0,0),thickness=1,)
#           for i in distances:
#               x1=int(face_landmarks.landmark[i[0]].x * width)
#               y1=int(face_landmarks.landmark[i[0]].y * height)
#               x2=int(face_landmarks.landmark[i[1]].x * width)
#               y2=int(face_landmarks.landmark[i[1]].y * height)
#               cv2.line(img=annotated_image,pt1=(x1,y1),pt2=(x2,y2),color=(255,0,0),thickness=2)
#     # Flip the image horizontally for a selfie-view display.
#     cv2.imshow('MediaPipe Face Mesh', cv2.flip(annotated_image, 1))
#     if cv2.waitKey(5) & 0xFF == 27:
#         cv2.destroyAllWindows()
#         break

In [8]:
# import time
# import numpy as np
# import joblib
#
# loaded_model = joblib.load("./fer2013_random_forest.joblib")
# drawing_spec = mp_drawings.DrawingSpec(thickness=1, circle_radius=1)
# cap = cv2.VideoCapture(0)
# data=[]
# emotions = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Neutral', 5:'Sad', 6:'Surprise'}
# with mp_face_mesh.FaceMesh(
#         max_num_faces=1,
#         refine_landmarks=True,
#         min_detection_confidence=0.5,
#         min_tracking_confidence=0.5) as face_mesh:
#     while cap.isOpened():
#         success, annotated_image = cap.read()
#         height,width,_= annotated_image.shape
#         if not success:
#             print("Ignoring empty camera frame.")
#             # If loading a video, use 'break' instead of 'continue'.
#             continue
#
#         # To improve performance, optionally mark the image as not writeable to
#         # pass by reference.
#         annotated_image.flags.writeable = False
#         annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
#         results = face_mesh.process(annotated_image)
#
#         # Draw the face mesh annotations on the image.
#         annotated_image.flags.writeable = True
#         annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)
#         if results.multi_face_landmarks:
#             for face_landmarks in results.multi_face_landmarks:
#                 data = []
#                 for i in distances:
#                     x1=int(face_landmarks.landmark[i[0]].x * width)
#                     y1=int(face_landmarks.landmark[i[0]].y * height)
#                     x2=int(face_landmarks.landmark[i[1]].x * width)
#                     y2=int(face_landmarks.landmark[i[1]].y * height)
#                     data.append(calculate_distance(face_landmarks.landmark[i[0]],face_landmarks.landmark[i[1]],height))
#                     time.sleep(3)
#             print(data)
#             if loaded_model.predict(np.array(data).reshape((1, -1)))[0].tolist().__contains__(1):
#                 cv2.putText(img=annotated_image,text=emotions[loaded_model.predict(np.array(data).reshape((1, -1)))[0].tolist().index(1)],org=(100,100),fontFace=cv2.FONT_HERSHEY_DUPLEX,fontScale=0.3,color=(255,0,0),thickness=1,)
#             else:
#                 cv2.putText(img=annotated_image,text='PLEASE WAIT',org=(100,100),fontFace=cv2.FONT_HERSHEY_DUPLEX,fontScale=0.3,color=(255,0,0),thickness=1,)
#         # Flip the image horizontally for a selfie-view display.
#         cv2.imshow('MediaPipe Face Mesh', cv2.flip(annotated_image, 1))
#         if cv2.waitKey(5) & 0xFF == 27:
#             cv2.destroyAllWindows()
#             break
# cap.release()

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import joblib

data = []
loaded_model = joblib.load("./fer2013_model.joblib")
# emotions = {0:'NO', 1:'YES'}
emotions = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Neutral', 5:'Sad', 6:'Surprise'}
drawing_spec = mp_drawings.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(
        max_num_faces=2,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, annotated_image = cap.read()
        height,width,_= annotated_image.shape
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        annotated_image.flags.writeable = False
        annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(annotated_image)

        # Draw the face mesh annotations on the image.
        annotated_image.flags.writeable = True
        annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                data = []
                for i in DISTANCES:
                    x1=int(face_landmarks.landmark[i[0]].x * width)
                    y1=int(face_landmarks.landmark[i[0]].y * height)
                    x2=int(face_landmarks.landmark[i[1]].x * width)
                    y2=int(face_landmarks.landmark[i[1]].y * height)
                    # cv2.line(img=annotated_image,pt1=[x1,y1],pt2=[x2,y2],color=(0,0,255),thickness=1)
                    data.append(calculate_distance(face_landmarks.landmark[i[0]],face_landmarks.landmark[i[1]]))
                prediction = loaded_model.predict(np.array(data).reshape((1, -1)))[0]
                emotion_image = cv2.imread(os.path.join('../DATA/Assests/',emotions[prediction]+'.png'))
                emotion_image_size = 50
                emotion_image = cv2.resize(emotion_image, (emotion_image_size, emotion_image_size))
                emotion_image_gray = cv2.cvtColor(emotion_image, cv2.COLOR_BGR2GRAY)
                ret, mask = cv2.threshold(emotion_image_gray, 1, 255, cv2.THRESH_BINARY)
                roi = annotated_image[-emotion_image_size-10:-10, -emotion_image_size-10:-10]

                roi[np.where(mask)] = 0
                roi += emotion_image
                cv2.putText(img=annotated_image,text=emotions[prediction],org=(int(face_landmarks.landmark[UPPER_REF].x * width)-20,int(face_landmarks.landmark[UPPER_REF].y * width)-30),fontFace=cv2.FONT_HERSHEY_DUPLEX,fontScale=1,color=(255,0,0),thickness=1,)

        cv2.imshow('MediaPipe Face Mesh', annotated_image)
        if cv2.waitKey(5) & 0xFF == 27:
            cv2.destroyAllWindows()
            break